# 5 Uploading Model files

Demonstrates uploading a file (rather than registering a URL). This requires multiple steps, first creating content blob "placeholders", and then updating the content blobs with the data to be uploaded.

In [1]:
import requests
import json
import string

In [2]:
base_url = 'http://0.0.0.0:3000'

Set up the headers and authenticate just as in the earlier steps.

In [3]:
headers = {
    "Accept": "application/json",               # Specifies the client expects the response in JSON format.
    "Content-Type": "application/json",         # Informs the server that the request body is in JSON format.
    "Authorization": "Token {your_token_created_in_your_SEEK_instance}"  # Provides the API token for authentication.
}

In [4]:
def json_for_resource(type, id):
    url = f"{base_url}/{type}/{id}"
    response = requests.get(url, headers=headers)
    
    if response.status_code != 200:
        print(response.json())
    
    response.raise_for_status()
    return response.json()

Define the projects id.

In [5]:
containing_project_id = 1 

This defines the placeholder information for the content blob. For now just the filename and mime type are provided.

In [6]:
local_blob = {'original_filename' : 'Copasi_model_file.xml', 'content_type' : 'text/xml'}

Set up the attributes for the model and include the content blob details. This also shows the intent that content blobs will be provided.

This code constructs a Python dictionary named model that is formatted to match the expected structure of a JSON payload for a POST request.

In [7]:
model = {}
model['data'] = {}
model['data']['type'] = 'models'
model['data']['attributes'] = {}
model['data']['attributes']['title'] = 'Copasi model file'
model['data']['attributes']['policy'] = {'access':'download'}
model['data']['relationships'] = {}
model['data']['relationships']['projects'] = {}
model['data']['relationships']['projects']['data'] = [{'id' : containing_project_id, 'type' : 'projects'}]

model['data']['attributes']['content_blobs'] = [local_blob]

Register the Model. The resulting JSON contains the content blob element, but this is currently blank. 

In [8]:
response = requests.post(base_url + '/models', headers=headers, json=model)
print(f"Response code: {response.status_code}")

populated_model = response.json()
print(json.dumps(populated_model, indent=4))

Response code: 200
{
    "data": {
        "id": "20",
        "type": "models",
        "attributes": {
            "policy": {
                "access": "download",
                "permissions": []
            },
            "discussion_links": [],
            "title": "Copasi model file",
            "license": null,
            "description": null,
            "latest_version": 1,
            "tags": [],
            "versions": [
                {
                    "version": 1,
                    "revision_comments": null,
                    "url": "http://localhost:3000/models/20?version=1",
                    "doi": null
                }
            ],
            "version": 1,
            "revision_comments": null,
            "created_at": "2024-12-02T14:31:33.000Z",
            "updated_at": "2024-12-02T14:31:33.000Z",
            "doi": null,
            "content_blobs": [
                {
                    "original_filename": "Copasi_model_file.xml",
            

The entry is now visible in Seek, but there is no content yet.

Now upload the file. Normally it would come from a local file, but we simulate this here by fetching from a URL (notice how the SEEK API is being used to fetch a copasi model file!)

*data* contains the data of a xml file



In [9]:
url = 'https://fairdomhub.org/models/730/content_blobs/10448/download'
response = requests.get(url)
data = response.content
print(data)

b'<?xml version="1.0" encoding="UTF-8"?>\n<sbml xmlns="http://www.sbml.org/sbml/level3/version1/core" level="3" version="1">\n  <model id="U2020_2" name="U2020_2" substanceUnits="substance" timeUnits="time_unit" volumeUnits="volume" areaUnits="area" lengthUnits="length" extentUnits="extent">\n    <listOfUnitDefinitions>\n      <unitDefinition id="substance" name="nM (default)">\n        <listOfUnits>\n          <unit kind="mole" exponent="1" scale="-9" multiplier="1"/>\n        </listOfUnits>\n      </unitDefinition>\n      <unitDefinition id="extent">\n        <listOfUnits>\n          <unit kind="mole" exponent="1" scale="-9" multiplier="1"/>\n        </listOfUnits>\n      </unitDefinition>\n      <unitDefinition id="time_unit" name="hour (default)">\n        <listOfUnits>\n          <unit kind="second" exponent="1" scale="3" multiplier="3.6"/>\n        </listOfUnits>\n      </unitDefinition>\n      <unitDefinition id="area">\n        <listOfUnits>\n          <unit kind="metre" expone

Now with a **PUT** update the content blob with the data. The url for the content blob was provided by the Model creation JSON response.

The **headers={'Content-Type': 'application/octet-stream'}** is currently required, otherwise in this case SEEK tries to process it as an invalid XML request resulting i an error. it indicates that the content is raw binary data

In [11]:
content_blob = populated_model['data']['attributes']['content_blobs'][0]
content_blob

{'original_filename': 'Copasi_model_file.xml',
 'url': None,
 'md5sum': None,
 'sha1sum': None,
 'content_type': 'text/xml',
 'link': 'http://localhost:3000/models/20/content_blobs/76',
 'size': None}

In [12]:
blob_url = content_blob['link']

headers["Content-Type"] = "application/octet-stream"

response = requests.put(blob_url, headers=headers, data=data)
print(response.status_code)

print(response.json)

200
<bound method Response.json of <Response [200]>>


Fetch the JSON for the model again, and look at the content blob attributes.

The size and checksums have now been updated with the new content.

In [13]:
model_json = json_for_resource('models',populated_model['data']['id'])
print(json.dumps(model_json, indent=4))

{
    "data": {
        "id": "20",
        "type": "models",
        "attributes": {
            "policy": {
                "access": "download",
                "permissions": []
            },
            "discussion_links": [],
            "title": "Copasi model file",
            "license": null,
            "description": null,
            "latest_version": 1,
            "tags": [],
            "versions": [
                {
                    "version": 1,
                    "revision_comments": null,
                    "url": "http://localhost:3000/models/20?version=1",
                    "doi": null
                }
            ],
            "version": 1,
            "revision_comments": null,
            "created_at": "2024-12-02T14:31:33.000Z",
            "updated_at": "2024-12-02T14:31:33.000Z",
            "doi": null,
            "content_blobs": [
                {
                    "original_filename": "Copasi_model_file.xml",
                    "url": null